In [22]:
import nltk,re,string,random
from nltk.tag import pos_tag
from nltk.corpus import stopwords,twitter_samples
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from nltk import FreqDist,classify,NaiveBayesClassifier

In [4]:
pos_tweets = twitter_samples.strings("positive_tweets.json")
neg_tweets = twitter_samples.strings("negative_tweets.json")
text = twitter_samples.strings('tweets.20150430-223406.json')
tok_tweets = twitter_samples.tokenized("positive_tweets.json")

In [5]:
len(text)

20000

In [6]:
len(pos_tweets)

5000

In [8]:
len(neg_tweets)

5000

In [9]:
print(tok_tweets[0])

['#FollowFriday', '@France_Inte', '@PKuchly57', '@Milipol_Paris', 'for', 'being', 'top', 'engaged', 'members', 'in', 'my', 'community', 'this', 'week', ':)']


In [11]:
print(pos_tag(tok_tweets[0]))

[('#FollowFriday', 'JJ'), ('@France_Inte', 'NNP'), ('@PKuchly57', 'NNP'), ('@Milipol_Paris', 'NNP'), ('for', 'IN'), ('being', 'VBG'), ('top', 'JJ'), ('engaged', 'VBN'), ('members', 'NNS'), ('in', 'IN'), ('my', 'PRP$'), ('community', 'NN'), ('this', 'DT'), ('week', 'NN'), (':)', 'NN')]


In [12]:
lemmatizer = WordNetLemmatizer()

In [14]:
def lemmatize(txt):
    lemmatized = []
    for w,t in pos_tag(txt):
        if t.startswith("NN"):
            pos = 'n'
        elif t.startswith("VB"):
            pos = 'v'
        else:
            pos = 'a'
        lemmatized.append(lemmatizer.lemmatize(w,pos))
    return lemmatized

lemmatize(tok_tweets[0])

['#FollowFriday',
 '@France_Inte',
 '@PKuchly57',
 '@Milipol_Paris',
 'for',
 'be',
 'top',
 'engage',
 'member',
 'in',
 'my',
 'community',
 'this',
 'week',
 ':)']

In [25]:
def remove_noise(tweets,stopwords=()):
    cleaned = []
    for tok,t in pos_tag(tweets):
        tok = re.sub("http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+#]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+",'',tok)
        tok = re.sub("(@[A-Za-z0-9_]+)","",tok)
        if t == 'NN':
            pos = 'n'
        elif t == "VB":
            pos = 'v'
        else:
            pos = 'a'
        tok = lemmatizer.lemmatize(tok,pos)
        if len(tok) > 0 and tok not in string.punctuation and tok.lower() not in stopwords:
            cleaned.append(tok.lower())
    return cleaned

In [26]:
stop_words = stopwords.words("english")

In [27]:
print(remove_noise(tok_tweets[0],stop_words))

['#followfriday', 'top', 'engaged', 'members', 'community', 'week', ':)']


In [28]:
s = "https://www.libgen.is/search.php?req=mastry&lg_topic=libgen&open=0&view=simple&res=25&phrase=1&column=def"
re.findall("http[s]://(?:[a-zA-Z]|[0-9]|[$-_@.&+#]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+",s)

['https://www.libgen.is/search.php?req=mastry&lg_topic=libgen&open=0&view=simple&res=25&phrase=1&column=def']

In [29]:
pos_tokens = twitter_samples.tokenized('positive_tweets.json')
neg_tokens = twitter_samples.tokenized('negative_tweets.json')
pos_cleaned_tok = []
neg_cleaned_tok = []
for tokens in pos_tokens:
    pos_cleaned_tok.append(remove_noise(tokens, stop_words))

for tokens in neg_tokens:
    neg_cleaned_tok.append(remove_noise(tokens, stop_words))

In [30]:
print(pos_tokens[500])
print(pos_cleaned_tok[500])

['Dang', 'that', 'is', 'some', 'rad', '@AbzuGame', '#fanart', '!', ':D', 'https://t.co/bI8k8tb9ht']
['dang', 'rad', '#fanart', ':d']


In [36]:
def get_all_words(cleaned_tokens_list):
    for tokens in cleaned_tokens_list:
        for token in tokens:
            yield token

all_pos_words = get_all_words(pos_cleaned_tok)

In [37]:
freq_dist_pos = FreqDist(all_pos_words)
print(freq_dist_pos.most_common(10))

[(':)', 3691), (':-)', 701), (':d', 658), ('thanks', 391), ('follow', 304), ('...', 290), ('good', 283), ('love', 274), ('thank', 248), ('u', 245)]


In [38]:
def get_tweets_for_model(cleaned_tokens_list):
    for tweet_tokens in cleaned_tokens_list:
        yield dict([token, True] for token in tweet_tokens)

positive_tokens_for_model = get_tweets_for_model(pos_cleaned_tok)
negative_tokens_for_model = get_tweets_for_model(neg_cleaned_tok)

In [39]:
import random

positive_dataset = [(tweet_dict, "Positive")
                     for tweet_dict in positive_tokens_for_model]

negative_dataset = [(tweet_dict, "Negative")
                     for tweet_dict in negative_tokens_for_model]

dataset = positive_dataset + negative_dataset

random.shuffle(dataset)

train_data = dataset[:7000]
test_data = dataset[7000:]

In [40]:
classifier = NaiveBayesClassifier.train(train_data)

print("Accuracy is:", classify.accuracy(classifier, test_data))

print(classifier.show_most_informative_features(10))

Accuracy is: 0.9953333333333333
Most Informative Features
                      :( = True           Negati : Positi =   2055.1 : 1.0
                      :) = True           Positi : Negati =    999.5 : 1.0
                     sad = True           Negati : Positi =     52.2 : 1.0
                     bam = True           Positi : Negati =     21.7 : 1.0
                 arrived = True           Positi : Negati =     17.0 : 1.0
                     x15 = True           Negati : Positi =     15.6 : 1.0
                   loves = True           Positi : Negati =     14.6 : 1.0
                 welcome = True           Positi : Negati =     12.8 : 1.0
                     via = True           Positi : Negati =     12.6 : 1.0
               wonderful = True           Positi : Negati =     12.4 : 1.0
None


In [41]:
custom_tweet = "I ordered just once from TerribleCo, they screwed up, never used the app again."

custom_tokens = remove_noise(word_tokenize(custom_tweet))

print(classifier.classify(dict([token, True] for token in custom_tokens)))

Negative


In [42]:
custom_tweet = 'Thank you for sending my baggage to CityX and flying me to CityY at the same time. Brilliant service. #thanksGenericAirline'
custom_tokens = remove_noise(word_tokenize(custom_tweet))

print(classifier.classify(dict([token, True] for token in custom_tokens)))

Positive
